In [40]:
# Imports
import csv
import glob2
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import scanpy as sc
import squidpy as sq
import warnings
from anndata import AnnData
from scipy import sparse

In [5]:
# Globals
PRJ_DIR = "/scratch/gpfs/KANG/sereno/spatialstem"
SRC_DIR = f"{PRJ_DIR}/sourcefiles"
RAW_DIR = f"{SRC_DIR}/raw"
HAD_DIR = f"{SRC_DIR}/h5ad"
INT_DIR = f"{PRJ_DIR}/intermediates"
FIG_DIR = f"{PRJ_DIR}/figs"

In [24]:
# Dann: Mapping the developing human immune system across organs
dann_dir = f"{RAW_DIR}/pub1_dann"
dann_paths = glob2.glob(f"{dann_dir}/*")
# Already formatted...

In [53]:
# Niec: Lymphatics act as a signaling hub to regulate intestinal stem cell activity
niec_dir = f"{RAW_DIR}/pub2_niec"
niec_paths = glob2.glob(f"{niec_dir}/*")
niec_paths.sort()
niec_labels = ["largeintestine1", "largeintestine2", "smallintestine1", "smallintestine2"]
for niec_path, niec_label in zip(niec_paths, niec_labels):
    # Will warn you that your variable names aren't unique, fixed below.
    with warnings.catch_warnings(action="ignore"):
        visium_in = sc.read_visium(niec_path)
    visium_in.var_names_make_unique()
    h5_out_path = f"{HAD_DIR}/p2_{niec_label}.h5ad"
    visium_in.write_h5ad(h5_out_path)
    print(f"Written: {h5_out_path}")

Written: /scratch/gpfs/KANG/sereno/spatialstem/sourcefiles/h5ad/p2_largeintestine1.h5ad
Written: /scratch/gpfs/KANG/sereno/spatialstem/sourcefiles/h5ad/p2_largeintestine2.h5ad
Written: /scratch/gpfs/KANG/sereno/spatialstem/sourcefiles/h5ad/p2_smallintestine1.h5ad
Written: /scratch/gpfs/KANG/sereno/spatialstem/sourcefiles/h5ad/p2_smallintestine2.h5ad


In [49]:
# Calvanese: Mapping human hematopoietic stem cells from hemogenic endothelium to birth
calv_dir = f"{RAW_DIR}/pub3_calvanese"
calv_paths = glob2.glob(f"{calv_dir}/*")
calv_paths.sort()
for idx, calv_path in enumerate(calv_paths):
    with warnings.catch_warnings(action="ignore"):
        visium_in = sc.read_visium(calv_path)
    visium_in.var_names_make_unique()
    h5_out_path = f"{HAD_DIR}/p3_hsc{idx+1}.h5ad"
    visium_in.write_h5ad(h5_out_path)
    print(f"Written: {h5_out_path}")

['/scratch/gpfs/KANG/sereno/spatialstem/sourcefiles/raw/pub3_calvanese/HM-1', '/scratch/gpfs/KANG/sereno/spatialstem/sourcefiles/raw/pub3_calvanese/HM-2', '/scratch/gpfs/KANG/sereno/spatialstem/sourcefiles/raw/pub3_calvanese/HM-4', '/scratch/gpfs/KANG/sereno/spatialstem/sourcefiles/raw/pub3_calvanese/HM-5', '/scratch/gpfs/KANG/sereno/spatialstem/sourcefiles/raw/pub3_calvanese/HM-6', '/scratch/gpfs/KANG/sereno/spatialstem/sourcefiles/raw/pub3_calvanese/HM-7', '/scratch/gpfs/KANG/sereno/spatialstem/sourcefiles/raw/pub3_calvanese/HM-8']


In [90]:
# Larouche: Spatiotemporal mapping of immune and stem cell dysregulation after volumetric muscle loss
laro_dir = f"{RAW_DIR}/pub4_larouche"
laro_paths_raw = glob2.glob(f"{laro_dir}/*")
# Filters out raw RDS objects
laro_paths = [path for path in laro_paths_raw if 'GSE205707' not in path]
laro_paths.sort()
for laro_path in laro_paths:
    laro_lab = laro_path.split("/")[-1]
    visium_in = sc.read_10x_mtx(laro_path)
    coords_in = np.genfromtxt(f"{laro_path}/coords.csv", delimiter=",", dtype="int64")
    visium_in.obsm["spatial"] = coords_in
    h5_out_path = f"{HAD_DIR}/p4_{laro_lab}.h5ad"
    visium_in.write_h5ad(h5_out_path)
    print(f"Written: {h5_out_path}")

Written: /scratch/gpfs/KANG/sereno/spatialstem/sourcefiles/h5ad/p4_caD0IR1.h5ad
Written: /scratch/gpfs/KANG/sereno/spatialstem/sourcefiles/h5ad/p4_caD0IR2.h5ad
Written: /scratch/gpfs/KANG/sereno/spatialstem/sourcefiles/h5ad/p4_caD14E1.h5ad
Written: /scratch/gpfs/KANG/sereno/spatialstem/sourcefiles/h5ad/p4_caD14M1.h5ad
Written: /scratch/gpfs/KANG/sereno/spatialstem/sourcefiles/h5ad/p4_caD14M2.h5ad
Written: /scratch/gpfs/KANG/sereno/spatialstem/sourcefiles/h5ad/p4_caD7E1.h5ad
Written: /scratch/gpfs/KANG/sereno/spatialstem/sourcefiles/h5ad/p4_caD7M1.h5ad
Written: /scratch/gpfs/KANG/sereno/spatialstem/sourcefiles/h5ad/p4_caD7M2.h5ad
Written: /scratch/gpfs/KANG/sereno/spatialstem/sourcefiles/h5ad/p4_mmD141.h5ad
Written: /scratch/gpfs/KANG/sereno/spatialstem/sourcefiles/h5ad/p4_mmD142.h5ad
Written: /scratch/gpfs/KANG/sereno/spatialstem/sourcefiles/h5ad/p4_mmITD1.h5ad
Written: /scratch/gpfs/KANG/sereno/spatialstem/sourcefiles/h5ad/p4_mmITD2.h5ad
Written: /scratch/gpfs/KANG/sereno/spatialstem/

In [5]:
# # Error in this dataset (Shani, P5), deprecated for now.
# # Shani: The spatio-temporal program of liver zonal regeneration
# shan_dir = f"{RAW_DIR}/pub5_shani"
# shan_paths = glob2.glob(f"{shan_dir}/*.h5")
# shan_paths.sort()
# print(shan_paths)
# # Read metadata to get visium coords
# visium_meta_dict = {}
# with open(f'{shan_dir}/Visium_Meta_data.txt', mode ='r')as meta_file:
#     shan_meta = csv.reader(meta_file)
#     next(shan_meta)
#     for line in shan_meta:
#         sample = line[1].replace("t_", "")
#         x_coord = np.int64(line[4])
#         y_coord = np.int64(line[5])
#         coords = [x_coord, y_coord]
#         if sample not in visium_meta_dict:
#             visium_meta_dict[sample] = []
#         visium_meta_dict[sample].append(coords)
# # Notice that this does not work due to the visium_in not lining up with metadata.
# for shan_path in shan_paths:
#     # Extract mouse id
#     shan_lab = shan_path.split("/")[-1].replace("Visium_", "").replace("_raw_feature_bc_matrix.h5", "")
#     # Will warn you that your variable names aren't unique, fixed below.
#     with warnings.catch_warnings(action="ignore"):
#         visium_in = sc.read_10x_h5(shan_path)
#     visium_in.var_names_make_unique()
#     coords = np.asarray(visium_meta_dict[shan_lab])
#     visium_in.obsm["spatial"] = coords
#     h5_out_path = f"{HAD_DIR}/p5_{shan_lab}.h5ad"
#     # visium_in.write_h5ad(h5_out_path)

['/scratch/gpfs/KANG/sereno/spatialstem/sourcefiles/raw/pub5_shani/Visium_24h_m1_raw_feature_bc_matrix.h5', '/scratch/gpfs/KANG/sereno/spatialstem/sourcefiles/raw/pub5_shani/Visium_24h_m2_raw_feature_bc_matrix.h5', '/scratch/gpfs/KANG/sereno/spatialstem/sourcefiles/raw/pub5_shani/Visium_48h_m4_raw_feature_bc_matrix.h5', '/scratch/gpfs/KANG/sereno/spatialstem/sourcefiles/raw/pub5_shani/Visium_48h_m5_raw_feature_bc_matrix.h5', '/scratch/gpfs/KANG/sereno/spatialstem/sourcefiles/raw/pub5_shani/Visium_72h_m1_raw_feature_bc_matrix.h5', '/scratch/gpfs/KANG/sereno/spatialstem/sourcefiles/raw/pub5_shani/Visium_72h_m2_raw_feature_bc_matrix.h5']


In [40]:
# Some example Shani objects: obj 1
shan_path = shan_paths[1]
shan_lab = shan_path.split("/")[-1].replace("Visium_", "").replace("_raw_feature_bc_matrix.h5", "")
# Will warn you that your variable names aren't unique, fixed below.
with warnings.catch_warnings(action="ignore"):
    visium_in = sc.read_10x_h5(shan_path)
visium_in.var_names_make_unique()
print(shan_path)
visium_in

/scratch/gpfs/KANG/sereno/spatialstem/sourcefiles/raw/pub5_shani/Visium_24h_m2_raw_feature_bc_matrix.h5


AnnData object with n_obs × n_vars = 4992 × 32285
    var: 'gene_ids', 'feature_types', 'genome'

In [41]:
# Some example Shani objects: obj 2
shan_path = shan_paths[2]
shan_lab = shan_path.split("/")[-1].replace("Visium_", "").replace("_raw_feature_bc_matrix.h5", "")
# Will warn you that your variable names aren't unique, fixed below.
with warnings.catch_warnings(action="ignore"):
    visium_in = sc.read_10x_h5(shan_path)
visium_in.var_names_make_unique()
print(shan_path)
visium_in

/scratch/gpfs/KANG/sereno/spatialstem/sourcefiles/raw/pub5_shani/Visium_48h_m4_raw_feature_bc_matrix.h5


AnnData object with n_obs × n_vars = 4991 × 32285
    var: 'gene_ids', 'feature_types', 'genome'

In [59]:
# TODO: Run overnight
# Biermann: Dissecting the treatment-naive ecosystem of human melanoma brain metastasis
# Index of count files and spatial info should be equivalent.
bier_dir = f"{RAW_DIR}/pub6_biermann"
bier_count_paths = glob2.glob(f"{bier_dir}/*raw_counts*")
bier_count_paths.sort()
bier_coord_paths = glob2.glob(f"{bier_dir}/*spatial_info*")
bier_coord_paths.sort()
for bier_count_path, bier_coord_path in zip(bier_count_paths, bier_coord_paths):
    counts = pd.read_csv(bier_count_path, compression='gzip', index_col=0).T
    coords = pd.read_csv(bier_coord_path, compression='gzip', usecols=['xcoord', 'ycoord'])
    bier_path_split = bier_count_path.split("/")[-1].split("_")
    sample_id = ("_").join(bier_path_split[1:]).replace("_slide_raw_counts.csv.gz", "")
    bier_ad = sc.AnnData(sparse.csr_matrix(counts), counts.index.to_frame(), counts.columns.to_frame())
    # Manual coordinate frame build
    coords_arr = np.asarray(coords)
    bier_ad.obsm["spatial"] = coords_arr
    # Manual feature info frame build.
    gene_names = counts.columns
    feature_type_rep = ["Gene Expression"] * len(gene_names)
    bier_gene_frame = pd.DataFrame(index=gene_names, data={"gene_ids": gene_names, "feature_types": feature_type_rep})
    bier_ad.var = bier_gene_frame
    # Manual obs frame build
    barcodes = counts.index
    bier_obs_frame = pd.DataFrame(index=barcodes, data={"barcode": barcodes})
    bier_obs_frame.index.name = "barcode"
    bier_ad.obs = bier_obs_frame
    h5_out_path = f"{HAD_DIR}/p6_{sample_id}.h5ad"
    bier_ad.write_h5ad(h5_out_path)
    print(f"Written: {h5_out_path}")

Written: /scratch/gpfs/KANG/sereno/spatialstem/sourcefiles/h5ad/p6_MBM05_rep1.h5ad


In [65]:
# TODO: Run overnight
# Rajachandran: Dissecting the spermatogonial stem cell niche using spatial transcriptomics
raja_dir = f"{RAW_DIR}/pub7_rajachandran"
raja_paths = glob2.glob(f"{raja_dir}/*")
for raja_path in raja_paths:
    raja_counts = glob2.glob(f"{raja_path}/*MappedDGE*")[0]
    raja_coords = glob2.glob(f"{raja_path}/*BeadLocations*")[0]
    counts = pd.read_csv(raja_counts, index_col=1)
    counts.drop("Unnamed: 0", axis=1, inplace=True)
    coords = pd.read_csv(raja_coords, usecols=['x', 'y'])
    sample_id = raja_path.replace(f"{raja_dir}/", "")
    raja_ad = sc.AnnData(sparse.csr_matrix(counts), counts.index.to_frame(), counts.columns.to_frame())
    # Manual coordinate frame build
    coords_arr = np.asarray(coords)
    raja_ad.obsm["spatial"] = coords_arr
    # Manual feature info frame build.
    gene_names = counts.columns
    feature_type_rep = ["Gene Expression"] * len(gene_names)
    raja_gene_frame = pd.DataFrame(index=gene_names, data={"gene_ids": gene_names, "feature_types": feature_type_rep})
    raja_ad.var = raja_gene_frame
    h5_out_path = f"{HAD_DIR}/p7_{sample_id}.h5ad"
    raja_ad.write_h5ad(h5_out_path)
    print(f"Written: {h5_out_path}")

Written: /scratch/gpfs/KANG/sereno/spatialstem/sourcefiles/h5ad/p7_diabetes3.h5ad


In [ ]:
# Chen: Human neural stem cells restore spatial memory in a transgenic Alzheimer's disease mouse model 
# by an immunomodulating mechanism
